# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [6]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:

    # Scroll down to the bottom.
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load the page.
    time.sleep(2)

    # Calculate new scroll height and compare with last scroll height.
    new_height = browser.execute_script("return document.body.scrollHeight")

    if new_height == last_height:

        break

    last_height = new_height
    


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [7]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [8]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 工程師轉加盟「巷弄手搖飲」　新創思維號召「文創市集」成社區中心

▲Blay創辦人CJ Chiang。（圖／記者姚惠茹攝）
記者姚惠茹／新北報導
位在板橋火車站附近的民權路202巷弄中，有一家「巷弄手搖飲」，老闆正是新創旅行交友app「Blay」的創辦人CJ Chiang，並用新創思維來號召「文創市集」，期望將「巷弄手搖飲」打造成社區中心，首場《枋橋202巷仔町迷你市集》就在4月5日登場。
Blay是一個旅行交友app，兼具即時航班動態推播功能，創辦人CJ Chiang因為熱愛旅遊、喜愛學習不同文化，甚至放棄百萬年薪的工程師工作，轉攻開發旅行交友app，時至今日，Blay在全球已經累積超過15萬會員，並在2018年獲得東京交通運輸新創高峰會的微軟大獎。
▲CJ Chiang轉加盟「巷弄手搖飲」。（圖／記者姚惠茹攝）
CJ Chiang表示，自己花一個月的時間就學會怎麼開珍珠奶茶店，並學習每個位子的大大小小事務，用最短的時間建立自已的團隊，還說雖然是加盟手搖飲料店，但卻是用新創思維來開店，因為自己想要做的不僅是飲料店，而是經營整個社區。
CJ Chiang進一步表示，這樣的思維源自於自己在英國生活多年的經驗，因為當地小小社區都會有餐飲店，化身為居民的活動中心，成為社區民眾聚集的場所，因此就希望可以在這個民權路202巷的巷弄之中，打造一個這樣的社區中心。
▲CJ Chiang用新創思維號召文創市集，並透過街坊鄰居票選熱門選項。（圖／記者姚惠茹攝）
CJ Chiang表示，透過街坊鄰居票選出甜點、飾品、小玩具的選項，剛好這個巷弄的居民其實都已經來到二代年輕人，因此很快也號召到文創市集團隊，並決定在4月5日舉辦首場《枋橋202巷仔町迷你市集》，未來每兩週都會舉辦一次文創市集，凝聚社區向心力。
CJ Chiang指出，這次文創市集邀請到《racer.co》，是創立於2018年的台灣街頭新創品牌，以復古、戶外、穿搭為出發點，注重圖案的設計排版和穿著舒適，追求創新的設計；以及《花譜》的花藝設計工作室，主要是乾燥花設計為主。
CJ Chiang進一步指出，小玩具邀請到《toytoybuy偷偷買玩具》，專門販售經典復古及搞怪玩具，像是

▲夏普1日宣布將在生產歐洲、印度和中國大陸的據點生產外科口罩。（圖／路透）
記者王曉敏／台北報導
新冠肺炎疫情全球大流行，各地抗疫醫療設備開始出現供不應求的情況，使許多原先並未生產醫療設備的製造商陸續宣布將投入防疫產品的生產。電子廠夏普（Sharp）1日宣布將在生產歐洲、印度和中國大陸的據點，生產外科口罩，以應對疫情突發的大量需求。
夏普上周公告，應日本政府的要求，該公司於2月28日決定生產口罩，並於3月24日開始生產，生產地點在夏普三重縣多氣郡多氣町工廠，初期日產量約為15萬片，未來目標日產能可達50萬片。據日經新聞報導，夏普社長戴正吳在本周三表示，將在歐洲、印度和中國大陸的據點生產外科口罩。
戴正吳在致員工的信件中表示，製作口罩可能「將成為一項長期、永續經營的事業。」至於這些海外據點開始生產的日期及規模，則將會於日後決定。報導指出，夏普此舉是母公司鴻海於2月份發起的醫療用口罩生產措施的延伸。
據陸媒報導，鴻海集團深圳龍華廠區自2月5日已開始導入口罩生產線。而上個月也有台灣網友發現，鴻海旗下的「鴻騰精密」已取得醫療器材許可證字號，將逐步量產醫療級口罩。
此前包括Giorgio Armani、運動品牌NIKE、美國職業棒球大聯盟球衣製造商Fanatics，以及義大利超跑廠Lamborghini（藍寶堅尼）等皆相繼投入生產口罩與防護衣等防疫產品的行列。而LVMH集團也動員旗下Dior、嬌蘭（Guerlain）、Parfums Givenchy（紀梵希）香水生產線，全力生產乾洗手。
----------------------------------------------------------------------

[8] 「疫情+沙俄石油戰」雙打擊　美國石油大廠凍未條...聲請破產

▲懷汀石油公司也受到新冠肺炎疫情影響。（示意圖／路透社）
實習記者曾筠淇／綜合報導
新冠肺炎疫情帶來了許多不確定性，加上沙烏地阿拉伯和俄羅斯所引發的價格戰持續太久，雙重打擊讓美國頁岩油大廠「懷汀石油公司」（Whiting Petroleum Corporation）淪為受害者，並向法院聲請破產保護，其股票也在紐約證券交易所暫停交易。
據美國商業雜誌《富比士》（Forbes），總部位於美國科羅拉多州（Colorado）丹佛市（Denver）的懷汀石油公司已向德州南區破產法院聲請第1

▲牌照稅4月起開徵，多家銀行除了提供一次繳清免手續費外，同時也提供分期繳稅方案。（示意圖／達志影像）
記者紀佳妘／台北報導
牌照稅4月起開徵，繳納期限到4月底為止，多家銀行除了提供一次繳清免手續費好康外，同時也提供刷卡分期0利率優惠，在新冠肺炎（COVlD-19）疫情尚未見遏止之勢，車主不妨考慮以信用卡線上完成繳稅，不僅能夠做到省時防疫，還能申請分期繳款的福利。
國泰世華銀行除了提供卡友一次付清免收手續費，同時也提供分期方案，可依個人需求選擇3、6、12、24期分期繳納，只要6月底前刷卡繳稅可享3、6期分期0利率；12、18、24期利率0.5%；到了今年7月1日後，一樣可分3、6、12、18、24期，利率0.5%。
台北富邦銀行則提供不限卡別，皆可申請3、6期0利率的繳牌照稅分期方案，此外富邦J卡及尊御世界卡的卡友還享有12期0利率的優惠活動；玉山銀行則推出不限卡別，均享6期0利率的繳稅方案。
台新銀行也提供一次付清免收手續費，繳牌照稅分期方案最高6期0利率；另外，街口聯名卡綁定街口支付APP繳納台北市牌照稅回饋6%、網路或電話繳稅回饋0.5%還可抽返還牌照稅額、全家、7-11等超商繳稅最高送50元購物金。
中信銀行則祭出刷任何一張信用卡繳牌照稅，皆享6期0利率，其中刷商旅鈦金卡，同享0.2%現金回饋無上限；元大銀行也有分期方案，民眾只要在網路銀行、行動銀行或e櫃檯申辦分期，享3期2.88%，6-30期5.88%分期優惠利率。
花旗銀行除了提供一次付清免手續費優惠外，想要選擇分期繳稅的民眾最高可分30期，分期利率在9.04%至15%之間，同時也推出滿福貸優惠方案，提供民眾另一項選擇。
----------------------------------------------------------------------

[20] 美股第2季開季不利！道瓊大跌973點　三大股重挫4.4％以上

▲美股第2季開季第1天，道瓊再度下跌近千點。（圖／路透社）
記者張靖榕／綜合外電報導
截至台灣時間清晨5點，新冠肺炎已蔓延全球205國或地區，突破92萬人確診、4.6萬人病逝，美國確診病例也已突破20萬人。投資人持續恐慌，美股第2季開盤就不利，道瓊指數再重挫973點。
道瓊工業指數大跌973.65點，或4.44%，收在20943.51點；標準普爾指數跌114.09點，或4

▲美國申請失業救濟人數恐再創新高。（圖／路透）
記者謝仁傑／綜合報導
根據路透報導，因美國推出新刺激法案，使自營業者也能獲得失業救濟，預估3月28日的一週之中，初請失業金人數可能攀升至350萬人、不僅超越上周的328萬人前高，最悲觀可能達525萬人。
美國總統川普上周五簽署的2.2兆美元《新冠病毒援助、救濟和經濟安全法案》（CARES），讓勞工更容易獲得失業救濟，每位勞工每周的失業救濟金最多提高了600美元（約新台幣18,170元），最多可獲得四個月的救濟。
凱投宏觀（Capital Economics）資深經濟學家 Michael Pearce示警，這項刺激措施會產生一些負面效應，有可能反而使失業人數膨脹，在每周額外增加600美元的聯邦失業保障，換算下來，等於每周40小時、每小時15美元，反而使勞工們更願意接受臨時裁員。
----------------------------------------------------------------------

[33] 財訊／從不參與家族事業的二姊出手　中環拿下台南企業最大股權幕後

文／林苑卿
2月26日傍晚，光碟片大廠中環發布公告，2019年4月1日至2020年2月26日，中環以每單位27.20元，總計以約3.02億元，取得台南企業7.388%股權。然而，事實上根據統計，截至2月24日，中環及子公司中嘉國際投資總計已持有台南企業高達12.33%股權，已成為台南企業最大股東。
攤開台南企業的前10大股東與董事會名單，董事長楊青峰及兩位姊姊、3位妹妹，歷年來就僅有2姊楊麗容從未列名其中，巧的是楊麗容正是中環董事長翁明顯的夫人。
手足翻臉？ 2姊突然冒出頭
如今，中環卻花了約1年的時間，以自有資金從集中市場取得台南企業的股權，還買到成為第一大股東。據消息人士透露，這背後可能牽涉的是家族內訌，甚至可能導致經營權之爭。
楊青峰有兩位姊姊、3位妹妹，大姊楊乃蕙在台南企業是大股東保羅投資法人代表監察人，兒子蘇炯忠是信望愛投資法人代表董事。大妹楊惠馨的夫婿吳道昌是台南企業總經理暨大股東良道投資法人代表董事，也是公司老臣。2妹楊媖媖、3妹楊富琴，也分別是大股東恬美投資和日富投資的法人代表董事。
董事會中，還有楊青峰的夫人王芳瓊是景達開發和大股東景宗投資負責人，也擔任董事，使台南企業成為台股裡典型的家族企業代表。
儘管從財報來

▲鴻海土城總部。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
鴻海（2317）今（1）日宣布，入選《Cheers》雜誌「2020新世代最嚮往企業調查TOP100」第4名，成為職場新鮮人的「理想頭家」，並宣布受新冠肺炎（COVID-19）疫情影響，開設徵才社群網站Linkedln企業專頁、LINE官方帳號、線上履歷登入等，全面啟動線上徵才。
鴻海表示，為有效整合海內外人才招募，鴻海今年3月啟用Linkedln企業專頁，透過網路社群平台形式，建立對外溝通多元管道，並祭出百萬年薪，瞄準電動車（EV）、數位健康（Digital Health）、機器人（Robotics）等三大未來產業方向，以及人工智慧（AI）、半導體（Semiconductor）及下世代行動通訊技術，三大關鍵核心技術領域關鍵人才，為集團下階段成長建立人才庫。
鴻海指出，2020年校園徵才場次受新冠肺炎影響，在不影響職場新鮮人的職涯規劃為前提，鴻海首次以線上徵才方式搶先起跑，掃瞄QR Code即可在線上填寫簡歷，若應聘人獲得應徵通知，後續面試與面談皆可透過線上管道進行，並在徵才專屬LINE官方帳號也更新相關徵才資訊。
鴻海說明，為讓員工未來家庭規劃無後顧之憂，在個人職場盡情發揮，勇於挑戰自我，鴻海在健康與福利方面同時有一系列完整規劃，首先是優渥薪資待遇，大學畢業生起薪4.5萬元以上、碩士畢業生起薪5.2萬元以上、博士畢業生起薪6萬元以上，鼓勵優秀人才加入並挑戰百萬以上年薪。
鴻海補充，針對員工眷屬方面，透過「鴻海人健康照護網」計劃照護鴻海人的下一代，今年宣布鴻海法人旗下員工將加碼0到3歲的育兒獎勵金擴大到6歲（含未滿7歲），只要符合條件的員工，在寶寶呱呱墜地之後，每個月可以領到1.5萬元，讓公司一路協助每位鴻海寶寶成長到滿7歲為止。
鴻海強調，除了員工、眷屬照護之外，鴻海內部規劃辦理各類專業、管理、通識課程及一系列的講堂活動，提供完善培訓機制及職涯發展機會，並針對三大產業及三大核心技術提供充分培訓，透過多元靈活的戰略培訓計劃，讓每個人能在數位化浪潮中茁壯成長為新世代領導者。
----------------------------------------------------------------------

[45] 遠航張綱維掏空22億獲800萬交保　檢方抗告成功撤銷發回

▲張綱維獲800萬交保，高

▲ 台灣證券交易所。（圖／記者李瑞瑾攝）
記者李瑞瑾／綜合報導
證交所表示，為搭建臺灣優質企業與國際機構投資人之間更即時有效的溝通橋樑，與寬量國際財務顧問公司（QIC）、群益金鼎證券將於4月6至10日共同舉辦「Taiwan CEO Week On Air」線上臺灣投資說明會，首度採用連線參與方式，讓33家與會的臺灣上市、上櫃及興櫃公司CEO，能在防疫期間超越地域限制，暢通無礙與國際機構投資人對談。
證交所每年於國內及國際重要金融中心舉辦臺灣投資說明會，向國際法人推介臺股優勢與投資價值。過去國際法人偏好與投資對象面對面交流，因此說明會皆以實地進行。近期受到新冠肺炎疫情影響，眾多實體會議取消或延期，連帶影響我國上市櫃公司與國際法人交流機會，對中小型公司而言，衝擊更為明顯。此回證交所聯合業者以創新數位方式舉辦一連五日的大型線上投資說明會，不但匯集我國半導體、電子、傳產及金融產業上市櫃公司，也一舉擴大邀請全球機構投資人共襄盛舉。
投資說明會期間，除安排上市櫃公司與全球機構投資人進行會談外，亦將舉辦線上實況專題演講，由證交所及國際財經專家向與會投資人說明臺股投資優勢。
證交所表示，高度重視各方市場參與者的權益，即使在防疫期間，市場服務亦未曾中斷，反而藉此機會，檢視如何運用科技降低資本市場活動成本，全力維護我國上市公司與投資人權益。目前證交所已規劃完整線上服務配套方案，自今年二月以來，陸續舉辦線上公司治理評鑑宣導課程，便利上市公司了解我國公司治理評鑑制度最新發展；並協助上市公司舉辦線上法說會，讓國內投資人可透過WebPro 3.0影音傳播網轉播取得公司最新財務業務資訊，國內市場溝通順暢無虞。
本次進一步舉辦線上臺灣投資說明會，是證交所服務上市公司及海外機構法人的一大創新，對部分較需支援國際行銷的中小型企業而言，更是重要的工具資源，預計可協助眾多上市櫃公司聯合向國際招手。
----------------------------------------------------------------------

[57] 涓豆腐攜手莫凡彼、鬍鬚張推外賣　餐飲八強推「一周不出門」方案自救

▲涓豆腐、莫凡彼、鬍鬚張等八強餐飲集團今（1）日推出聯名外賣方案搶市。（圖／記者林淑慧攝）
記者林淑慧／台北報導
新冠肺炎（COVID-19）疫情帶動自煮風潮，涓豆腐、莫凡彼、鬍鬚張等八強餐飲

▲翔宇旗下美容保養品「DV TOKYO」。（圖／翔宇提供）
記者姚惠茹／台北報導
翔宇（6539）董事會決議擬配發每股2.3元股票股利與每股2元現金股利，合計配發每股4.3元股利，以今（1）日平盤價格44.6元計算，現金殖利率達4.48％，同時宣布實施庫藏股，預計買回500張，買回區間價31~61元。
翔宇表示，為激勵員工及提升員工向心力，董事會決議通過實施庫藏股，買回公司股份並轉讓予員工，預計自4月1日起至5月31日，買回自家股票500張，買回價格區間為31~61元，且當股價低於所訂區間價格下限時將繼續買回。
展望2020年上半年，翔宇指出，雖因新冠肺炎疫情的延燒使得消費者至藥妝通路、DV皮膚管理中心等實體通路的消費力道減緩，但在積極優化整體營運表現，藉由強化線上電商平台的經營，期能推升來自DV官網、Momo購物網等的銷售業績增加。
翔宇強調，未來將持續升級產品配方，預計今年第2季將陸續推出2款訴求增加新陳代謝與養顏美容等的保健食品，可望因新品推出帶動保健品營收逐步向上墊高，有信心今年整體營運交出下半年優於上半年的成長態勢。
----------------------------------------------------------------------

[71] 影／聯邦銀暖心挺第一線醫護人員　刷卡最高回饋900點

記者紀佳妘／台北報導
新冠肺炎（COVID-19）疫情蔓延，醫療人員壓力日益加重，為鼓勵第一線防疫人員用心守護家園，聯邦銀行率先發起第一波「大力挺醫護」專案，最高致贈LINE POINTS 900點（等同900元），以實際行動回饋辛勞的醫護人員。
聯邦銀行信用卡暨支付金融處執行長熊令容表示，這次疫情來勢洶洶，醫護相關人員面對的風險與壓力有增無減，他們犧牲休假及休閒為全民抗疫，辛勞的付出有目共賭。為了感謝辛苦的醫護，聯邦銀行努力構思如何讓醫護人員能夠直接受用到這份心意，由於捐款必須有較複雜的分配流程，因此特別規劃這次的挺醫護專案。
「聯邦大力挺醫護」專案主要回饋辛勞的醫護人員，只要在衛福部登記的醫護人員都是這次的活動對象，即日起至6月底在活動官網登錄並上傳執業執照或證照後，以聯邦銀賴點卡每月任刷一筆，即致贈第一線醫護人員每月LINE POINTS 300點（等同300元），每卡最高致贈900點（等同900元），以實際行動支持辛勞的醫護人員

 ▲美國Cordish集團當年曾與和旺董事長劉永祥（右）開記者會宣稱合作。但劉永祥掏空公司，今遭判刑定讞。（圖／CORDISH集團提供）
記者吳銘峯／台北報導
和旺聯合實業前董事長劉永祥，因為爛賭而在澳門欠下鉅額賭債，沒想到他竟因此掏空公司50億元資產，還與多名股市炒手合作炒股坑殺散戶。一二審均將他判刑10年半，更一審改判9年，案經上訴，最高法院1日駁回上訴，劉永祥判刑9年定讞。
股票上櫃公司和旺聯合實業，從事建材營造業，股價最高時曾高達一股74元，但公司在2015年4月間爆發違約交割1億6千萬元，公司股價跳水式崩盤，腰斬一股不到30元。檢調單位調查後，發現有股市炒手刻意炒作和旺，經過深入調查，才發現是公司董事長劉永祥搞鬼。
檢調查出，劉永祥因爛賭而在澳門欠下鉅額賭債，竟先掏空公款5億元，再盜開和旺支票7億6000多萬元作為私人債務擔保，且僅收5億多元訂金，就將市價30億元土地轉手給陳姓金主等人。劉永祥除挪用5億元清償賭債外，並過著奢華的生活，和旺公司也因此損失近50億元，最終陸續跳票。
不僅如此，檢方還陸續查出劉永祥諸多犯行，他也與股市炒手合作，對外宣稱將與美商Cordish合作渡假村事業，讓股價原本每股18元多飆漲至歷史新高74元；未幾又宣告沒有跟Cordish合作，股價因此暴跌，最後違約交割情況，還跳票1億4000多萬元，害許多散戶遭坑殺面臨破產。檢調帶回劉永祥偵辦，和旺股票也在2015年7月間黯然下櫃。
劉永祥遭提起公訴，一二審均將他重判有期徒刑10年半，案經撤銷發回後，高院去年底的更一審，仍認定他犯《證券交易法》「操縱股價」、「非常規交易」、「特別背信」、「侵占」等罪，判刑9年。全案再上訴最高法院，最高法院1日駁回上訴定讞。
----------------------------------------------------------------------

[83] 無薪假來到308家、7916人　實施家數創2009年以來新高

▲勞動部公布最新無薪假數據。（圖／記者李毓康攝）
記者余弦妙／台北報導
勞動部今（1）日公布截至3月31日「因應景氣影響勞雇雙方協商減少工時」（俗稱無薪假）最新統計，實施家數308家，實施人數7,916人。較上期3月15日統計數據新增199家，實施人數新增4,081人。而這次的實施家數更是自2009年12月中以來最多。


▲勞動基金截至2月底跌幅達3.6％。（圖／記者李毓康攝）
記者余弦妙／台北報導
勞動部勞動基金運用局今（1）日公布勞動基金最新績效，其中，攸關勞工個人退休金帳戶的新制勞退基金，截至2月底收益數減少924.6億元，收益率下降3.74%，目前所有新制共計1,177萬名勞工，以目前收益數來看，每名勞工帳戶平均減少7,855元。
勞動基金2020年截至2月收益數達43,501億元，收益率下降3.6％，虧損1,514.4億元；其中攸關勞工個人退休金帳戶的新制勞退基金，收益減少924.6億元，收益率下降3.74%。
勞動基金局表示，截至2月底止，整體勞動基金規模為4兆7083億元，其中，新制勞退基金規模為2兆5001億元，舊制勞退基金規模為9492億元，勞保基金規模為7424億元，就保基金規模為1344億元，職災保護專款規模為110億元，積欠工資墊償基金規模為130億元。
根據統計，截至2月底整體勞動基金評價後收益數為-1,632.5億元，收益率3.58%。其中，新、舊制勞退基金、勞工保險基金、就保基金、職災保護專款及積欠工資墊償基金收益率分別為-3.74%、-4.02%、-3.47%、0.37%、0.15%及-0.3%。
另受衛生福利部委託管理的國民年金保險基金規模為3,582億元，收益數為-118.1億元，收益率為-3.35%。
勞動基金運用局表示，今(109)年2月份新冠肺炎（COVID-19）疫情持續擴大，全球產業鏈逐步受到衝擊，全球主要股票市場均大幅下跌。勞動基金積極掌控投資風險，降低市場震盪衝擊，尤其美國與歐洲等主要經濟體隨著疫情愈趨嚴峻，對全球經濟影響層面擴大，金融市場也隨著疫情擴散速度而大幅波動，震盪幅度創2008年金融海嘯以來紀錄，勞動基金短期績效受到影響，至2月底跌幅3.6％。
不過，勞動基金運用局也說，整體基金過去10年平均收益率4.16％，累積已幫全國勞工賺了1兆1,076億元。且勞動基金自108年下半年起，因考量股市已處相對高點，即已經逐步減緩股票投資部位，目前持股比重已低於原本的目標配置，有效降低市場震盪衝擊。加上新制勞工退休金依法有二年期定期存款之最低保證收益，勞工於退休時所領取之退休金，依法受到上開保障，不會因為短期間的評價損失而遭受到減損。
勞保局提醒，只要繳過新制勞退的勞工不管現在是否還有在繳納，都可以拿到分紅，只是拿到的分紅金額還要依照當時